In [7]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("../data/database.db")

query = "SELECT * FROM financial"
fin_df = pd.read_sql_query(query, conn)

conn.close()

fin_df.head()


,financial_id,company_id,date_of_publish,period_year,period_month,revenue_ttm,gross_profit_ttm,operating_profit_ttm,ebitda_ttm,net_income_ttm,...,current_assets,fixed_assets,long_term_debt,short_term_debt,gross_debt,net_debt,equity,eps_c,eps_q,eps_ttm
0,1,1,None,2022,3,NaN,NaN,NaN,NaN,NaN,...,1.989186e+10,1.765170e+10,7.358370e+09,2.108796e+10,8.842232e+09,6.496565e+09,9.097227e+09,2.0,2.0,NaN
1,2,1,None,2022,6,NaN,NaN,NaN,NaN,NaN,...,2.305158e+10,1.943500e+10,8.033575e+09,2.544772e+10,9.843746e+09,7.438998e+09,9.005287e+09,5.0,3.0,NaN
2,3,1,None,2022,9,NaN,NaN,NaN,NaN,NaN,...,2.703010e+10,2.123974e+10,8.793192e+09,2.869317e+10,1.074744e+10,7.298744e+09,1.078349e+10,8.0,3.0,NaN
3,4,1,None,2022,12,2.792529e+11,4.207056e+10,5.801538e+09,1.363232e+10,1.659943e+10,...,4.671791e+10,7.938305e+10,2.032255e+10,4.747106e+10,1.962585e+10,1.271510e+10,5.830735e+10,28.0,20.0,28.0
4,5,1,None,2023,3,3.480165e+11,5.163406e+10,4.009803e+09,1.459379e+10,1.650578e+10,...,3.597851e+10,4.005171e+10,1.250529e+10,3.719496e+10,1.454609e+10,1.051510e+10,2.632998e+10,2.0,2.0,28.0


In [ ]:
fin_df["current_ratio"] = fin_df["current_assets"] / fin_df["short_term_debt"]
fin_df["debt_to_equity"] = fin_df["gross_debt"] / fin_df["equity"]
fin_df["debt_to_assets"] = fin_df["gross_debt"] / (fin_df["fixed_assets"] + fin_df["current_assets"])
fin_df['debt_to_ebitda'] = fin_df['gross_debt'] / fin_df['ebitda_ttm']

fin_df["net_income_margin_ttm"] = fin_df["net_income_ttm"] / fin_df["revenue_ttm"]
fin_df["net_income_margin_q"] = fin_df["net_income_q"] / fin_df["revenue_q"]
fin_df["net_income_margin_c"] = fin_df["net_income_c"] / fin_df["revenue_c"]
fin_df["gross_profit_margin_ttm"] = fin_df["gross_profit_ttm"] / fin_df["revenue_ttm"]
fin_df["gross_profit_margin_q"] = fin_df["gross_profit_q"] / fin_df["revenue_q"]
fin_df["gross_profit_margin_c"] = fin_df["gross_profit_c"] / fin_df["revenue_c"]
fin_df["operating_margin_ttm"] = fin_df["operating_profit_ttm"] / fin_df["revenue_ttm"]
fin_df["operating_margin_q"] = fin_df["operating_profit_q"] / fin_df["revenue_q"]
fin_df["operating_margin_c"] = fin_df["operating_profit_c"] / fin_df["revenue_c"]

fin_df["roe_ttm"] = fin_df["net_income_ttm"] / fin_df["equity"]
fin_df["roe_q"] = fin_df["net_income_q"] / fin_df["equity"]
fin_df["roa_ttm"] = fin_df["net_income_ttm"] / (fin_df["fixed_assets"] + fin_df["current_assets"])
fin_df["roa_q"] = fin_df["net_income_q"] / (fin_df["fixed_assets"] + fin_df["current_assets"])
fin_df["roic_ttm"] = (fin_df['operating_profit_ttm'] * (1 - fin_df['effective_tax_rate_ttm'])) / fin_df['gross_debt'] + fin_df['equity']
fin_df['asset_turnover'] = fin_df['revenue_ttm'] / (fin_df["fixed_assets"] + fin_df["current_assets"])

fin_df = fin_df.sort_values(by=['company_id', 'period_year', 'period_month']).reset_index(drop=True)
fin_df["revenue_growth_ttm_yoy"] = (fin_df["revenue_ttm"] / fin_df.groupby('company_id')['revenue_ttm'].shift(4)) - 1
fin_df["revenue_growth_q_yoy"] = (fin_df["revenue_q"] / fin_df.groupby('company_id')['revenue_q'].shift(4)) - 1
fin_df["revenue_growth_q_qoq"] = (fin_df["revenue_q"] / fin_df.groupby('company_id')['revenue_q'].shift(1)) - 1
fin_df["net_income_growth_ttm_yoy"] = (fin_df["net_income_ttm"] / fin_df.groupby('company_id')['net_income_ttm'].shift(4)) - 1
fin_df["net_income_growth_q_yoy"] = (fin_df["net_income_q"] / fin_df.groupby('company_id')['net_income_q'].shift(4)) - 1
fin_df["net_income_growth_q_qoq"] = (fin_df["net_income_q"] / fin_df.groupby('company_id')['net_income_q'].shift(1)) - 1
fin_df["eps_growth_ttm_yoy"] = (fin_df["eps_ttm"] / fin_df.groupby('company_id')['eps_ttm'].shift(4)) - 1
fin_df["eps_growth_q_yoy"] = (fin_df["eps_q"] / fin_df.groupby('company_id')['eps_q'].shift(4)) - 1
fin_df["eps_growth_q_qoq"] = (fin_df["eps_q"] / fin_df.groupby('company_id')['eps_q'].shift(1)) - 1

ratio_df = fin_df[["company_id","period_year","period_month","date_of_publish","current_ratio","debt_to_equity","debt_to_assets","debt_to_ebitda","net_income_margin_ttm","net_income_margin_q","net_income_margin_c","gross_profit_margin_ttm","gross_profit_margin_q","gross_profit_margin_c","operating_margin_ttm","operating_margin_q","operating_margin_c","roe_ttm","roe_q","roa_ttm","roa_q","roic_ttm",'asset_turnover',"revenue_growth_ttm_yoy","revenue_growth_q_yoy","revenue_growth_q_qoq","net_income_growth_ttm_yoy","net_income_growth_q_yoy","net_income_growth_q_qoq","eps_growth_ttm_yoy","eps_growth_q_yoy","eps_growth_q_qoq"]]

ratio_df

,company_id,period_year,period_month,date_of_publish,current_ratio,debt_to_equity,debt_to_assets,net_income_margin_ttm,net_income_margin_q,net_income_margin_c,...,asset_turnover,revenue_growth_ttm_yoy,revenue_growth_q_yoy,revenue_growth_q_qoq,net_income_growth_ttm_yoy,net_income_growth_q_yoy,net_income_growth_q_qoq,eps_growth_ttm_yoy,eps_growth_q_yoy,eps_growth_q_qoq
0,1,2022,12,None,0.984135,0.336593,0.155636,0.059442,0.066772,0.059442,...,2.214519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022,3,None,0.943280,0.971970,0.235519,NaN,0.049213,0.049213,...,NaN,NaN,NaN,-0.845363,NaN,NaN,-0.886027,NaN,NaN,-0.900000
2,1,2022,6,None,0.905841,1.093107,0.231691,NaN,0.049317,0.049271,...,NaN,NaN,NaN,0.249743,NaN,NaN,0.252364,NaN,NaN,0.500000
3,1,2022,9,None,0.942040,0.996657,0.222653,NaN,0.043176,0.046826,...,NaN,NaN,NaN,0.206112,NaN,NaN,0.055941,NaN,NaN,0.000000
4,1,2023,12,None,1.007119,0.320295,0.150797,0.047026,0.062859,0.047026,...,2.217214,0.698104,-0.188655,2.480835,0.343403,-0.236203,4.067665,0.321429,-0.250000,4.000000
5,1,2023,3,None,0.967295,0.552454,0.191320,0.047428,0.013016,0.013016,...,4.577344,NaN,2.517527,-0.329582,NaN,-0.069671,-0.861178,NaN,0.000000,-0.866667
6,1,2023,6,None,0.927140,0.653543,0.204317,0.045270,0.039232,0.026484,...,4.965761,NaN,1.973467,0.056444,NaN,1.365427,2.184223,NaN,1.333333,2.500000
7,1,2023,9,None,0.976182,0.638609,0.192962,0.049427,0.060450,0.040168,...,5.104443,NaN,2.238013,0.313418,NaN,3.533466,1.023767,NaN,3.333333,0.857143
8,1,2024,12,None,0.991540,0.311932,0.156778,0.035824,0.032821,0.035824,...,2.202965,0.278360,0.665896,0.790823,-0.026156,-0.130166,-0.027670,-0.027027,-0.133333,0.000000
9,1,2024,3,None,1.026877,0.320612,0.142763,0.050640,0.037375,0.037375,...,2.758320,0.498294,0.491589,-0.399730,0.599757,3.282946,-0.316459,0.571429,3.500000,-0.307692
